# 内容
  - 妄想：機械学習によるコンプラ違反予想判定器

# 背景
  - 機械学習を調べていた
  - コンプライアンスの発表の順番が来た
  - 機械学習×コンプライアンス
    - コンプライアンス違反しそうな人を検知できそうだなぁと妄想してみた
    - 本気でやったらプロダクトが作れそうだなぁ

# 目的
  - コンプライアンスの発表を乗り切る
  - 雰囲気を感じてもらう
    - あー出来そうだねというのを感じでもらう
  - 何かの役に立てば

# 機械学習の流れ
  - サンプルデータを集め、きれいにする
  - モデルを選定
  - サンプルデータを学習用データと評価用データに分割
  - 学習用データで学習
  - 学習後、評価用データで評価
    - 答えが分かっているデータに対してどれだけ正答できるか
  - 問題なさそうなら実践投入：未知のデータの判定

# 妄想サンプルデータ
  - こんなデータがとれたとする(細かい事は気にしないでください)
    - 何らかの不正をしたかしてないかのフラグ
    - 業務に絡む人が少ない
    - 残業時間が多め
    - 面談ポイントが高い（上司採点のやばいんじゃない？というポイント数）
    - ストレス平均が高め
    - 退社時間が遅い
    - L3,L2のアクセス権があったりする


# で、実際にコードを動かしてみる
## Pythonのscikit-learnというライブラリを使います

In [ ]:
#必要なライブラリのロード
import pandas as pd
import numpy as np
%matplotlib inline

In [ ]:
#excelファイルからサンプルデータの読込
dframe = pd.read_excel('sample.xlsx',sheetname='OUT')

In [ ]:
#読み込んだ内容の表示
dframe

In [ ]:
#サンプルデータから結果を取り出す
y=np.array(dframe['不正有無0/1'])

In [ ]:
#サンプルデータからパラメータを取り出す
X=np.array(dframe[['面談ポイント★','残業時間平均','ストレスポイント★','金品関わり度合い★','主要業務関連人数','出社平均時間','退社平均時間','L3アクセス権有無','社有携帯登録件数','L2アクセス権有無']])

# k近傍法っていうモデルを学習させてみる

In [ ]:
from sklearn import neighbors
#モデルのインスタンス作成 
clf = neighbors.KNeighborsClassifier()

In [ ]:
#モデルの学習用のトレーニング用データと、評価用データに分割
from sklearn.model_selection import ShuffleSplit
ss = ShuffleSplit(train_size=0.8,test_size=0.2,random_state=1)
train_index, test_index = next(ss.split(X, y))
X_train, X_test = X[train_index], X[test_index]
y_train, y_test = y[train_index], y[test_index]

In [ ]:
#トレーニングデータで学習
clf.fit(X_train, y_train)

In [ ]:
#テストデータを食わせる。学習した結果どのぐらいの正答率となるか
clf.score(X_test,y_test)

# では新しいデータ(人)がコンプラ違反を起こしそうか？という事を判定させてみる

In [ ]:
#新しいデータの読み込み
dframe_new = pd.read_excel('sample.xlsx',sheetname='NewData')
X_new=np.array(dframe_new.values.tolist())

In [ ]:
#読み込んだ内容の表示
dframe_new

In [ ]:
#この人が違反しそうか予測してみる　結果：0:セーフ 1:ムムム！？
result = clf.predict(X_new)
result[0]

# 違うモデル:ニューラルネットワークでもやってみる

In [ ]:
# modified from https://gist.github.com/craffel/2d727968c3aaebd10359
import matplotlib.pyplot as plt
def draw_neural_net(layer_sizes, left=.1, right=.9, bottom=.1, top=.9, size=5):
    '''
    Draw a neural network cartoon using matplotilb.
    
    :usage:
        >>> fig = plt.figure(figsize=(12, 12))
        >>> draw_neural_net(fig.gca(), .1, .9, .1, .9, [4, 7, 2])
    
    :parameters:
        - ax : matplotlib.axes.AxesSubplot
            The axes on which to plot the cartoon (get e.g. by plt.gca())
        - left : float
            The center of the leftmost node(s) will be placed here
        - right : float
            The center of the rightmost node(s) will be placed here
        - bottom : float
            The center of the bottommost node(s) will be placed here
        - top : float
            The center of the topmost node(s) will be placed here
        - layer_sizes : list of int
            List of layer sizes, including input and output dimensionality
    '''
    fig = plt.figure(figsize=(size, size))
    ax = fig.gca()
    ax.axis('off')

    n_layers = len(layer_sizes)
    v_spacing = (top - bottom)/float(max(layer_sizes))
    h_spacing = (right - left)/float(len(layer_sizes) - 1)
    # Nodes
    for n, layer_size in enumerate(layer_sizes):
        layer_top = v_spacing*(layer_size - 1)/2. + (top + bottom)/2.
        for m in range(layer_size):
            circle = plt.Circle((n*h_spacing + left, layer_top - m*v_spacing), v_spacing/4.,
                                color='w', ec='k', zorder=4)
            ax.add_artist(circle)
    # Edges
    for n, (layer_size_a, layer_size_b) in enumerate(zip(layer_sizes[:-1], layer_sizes[1:])):
        layer_top_a = v_spacing*(layer_size_a - 1)/2. + (top + bottom)/2.
        layer_top_b = v_spacing*(layer_size_b - 1)/2. + (top + bottom)/2.
        for m in range(layer_size_a):
            for o in range(layer_size_b):
                line = plt.Line2D([n*h_spacing + left, (n + 1)*h_spacing + left],
                                  [layer_top_a - m*v_spacing, layer_top_b - o*v_spacing], c='k')
                ax.add_artist(line)

In [ ]:
#こんな感じのネットワーク。層の数、ノードの数は適当です。
draw_neural_net((10,5,2,1))
draw_neural_net((10,3,8,4,1))
draw_neural_net((10,7,3,6,4,1))

In [ ]:
from sklearn.neural_network import MLPClassifier
#いろいろなネットワークの形状でスコアを試して見る
for layers in [(10,5,2,1),
               (10,3,8,4,1),
               (10,7,3,6,4,1)]:
    clf = MLPClassifier(random_state=0)
    clf.hidden_layer_sizes = layers
    clf.fit(X_train, y_train)
    print("{0} hidden layers".format(len(layers)), clf.score(X_test, y_test))

# という訳で
今回は妄想のデータでしたがサンプルデータが取得ができ、不正しそうな**特徴がつかめれば**実現可能かも？
- しかし
  - 「コンプライアンス 機械学習」で検索してもそんなにヒットしない
  - http://itpro.nikkeibp.co.jp/atclact/active/14/368541/032400235/
  - 同じ事考えている人はいたが、少なそう
  - 人事データから特徴をつかむのは難しいのかもしれない
  - または、学習には大量のサンプルが必要だが少ないのかもしれない（不正する人のログって少ないですよね）

# 完